# Initial Setting

In [1]:
!pip install git+https://github.com/openai/CLIP.git
!pip install git+https://github.com/nghorbani/human_body_prior
!pip install --user ipykernel
!pip install h5py
!pip install smplx
!pip install chumpy
# !pip install matplotlib==3.4.3
# !pip install matplotlib-inline==0.1.2
# !pip install -r t2m_gpt_requirements.txt

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-7cfkvl9n
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-7cfkvl9n
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 1.2 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369497 sha256=2cf88ec0884550b7682585303312fa392ffffe0caa688ae467d1d23401deabaa
  Stored in directory: /tmp/pip-ephem-wheel-cache-y_i8ogdc/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip
  Cloning https://github.com/nghorbani/human_body_prior to /tmp/pip-req-build-5pmqkz25
  Running command git clone --filter=blob:none --quiet https://github.com/nghorbani/human_body_prior /tmp/pip-req-build-5pmqkz25
  Resolved https://github.com/nghorbani/human_body_prior to commit 4c246d8a8

In [2]:
!git clone https://github.com/Mael-zys/T2M-GPT.git
%cd /content/T2M-GPT/
!pwd

Cloning into 'T2M-GPT'...
remote: Enumerating objects: 153, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 153 (delta 34), reused 25 (delta 25), pack-reused 102
Receiving objects: 100% (153/153), 19.53 MiB | 22.76 MiB/s, done.
Resolving deltas: 100% (55/55), done.
/content/T2M-GPT
/content/T2M-GPT


In [3]:
# @title Connect to Google Drive
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
import sys, warnings, argparse, os, shutil, random
import tqdm, chumpy, smplx, h5py
import io, imageio
import matplotlib
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.axes3d as p3
import visualization.plot_3d_global as plot_3d
import numpy as np
import torch
import clip
import models.vqvae as vqvae
import models.t2m_trans as trans
import utils.rotation_conversions as geometry
import options.option_transformer as option_trans

from PIL import Image
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from textwrap import wrap
from tqdm import tqdm
from utils.motion_process import recover_from_ric
from visualize.joints2smpl.src import config
from visualize.joints2smpl.src.smplify import SMPLify3D
from models.rotation2xyz import Rotation2xyz
warnings.filterwarnings('ignore')

In [9]:
# @title Set argument
parser = argparse.ArgumentParser(description='Optimal Transport AutoEncoder training for Amass',
                                     add_help=True,
                                     formatter_class=argparse.ArgumentDefaultsHelpFormatter)

args = parser
args.dataname = 't2m'
args.resume_pth = 'pretrained/VQVAE/net_last.pth'
args.resume_trans = 'pretrained/VQTransformer_corruption05/net_best_fid.pth'
args.down_t = 2
args.depth = 3
args.block_size = 51
args.batch_size = 128
args.fps= [20]
args.seq_len = 64
args.total_iter = 100000
args.warm_up_iter = 1000
args.lr = 2e-4
args.lr_scheduler = [60000]
args.gamma = 0.05
args.weight_decay = 1e-6
args.decay_option = "all"
args.optimizer = "adamw"
args.code_dim = 512
args.nb_code = 512
args.mu = 0.99
args.stride_t = 2
args.width = 512
args.dilation_growth_rate = 3
args.output_emb_width = 512
args.vq_act = "relu"
args.embed_dim_gpt = 512
args.clip_dim = 512
args.num_layers = 2
args.n_head_gpt = 8
args.ff_rate = 4
args.drop_out_rate = 0.1
args.quantizer = "ema_reset"
args.quantbeta = 1.0
args.out_dir = 'output_GPT_Final/'
args.exp_name = "exp_debug"
args.vq_name = "exp_debug"
args.print_iter = 200
args.eval_iter = 5000
args.seed = 123
args.if_maxtest = "store_true"
args.pkeep = 1.0
args.seed = random.randint(1, 10)
print("Seed:", args.seed)

Seed: 8


In [10]:
## load clip model and datasets
clip_model, clip_preprocess = clip.load("ViT-B/32", device=torch.device('cuda'), jit=False, download_root='./')  # Must set jit=False for training
clip.model.convert_weights(clip_model)  # Actually this line is unnecessary since clip by default already on float16
clip_model.eval()
for p in clip_model.parameters():
    p.requires_grad = False

net = vqvae.HumanVQVAE(args, ## use args to define different parameters in different quantizers
                       args.nb_code,
                       args.code_dim,
                       args.output_emb_width,
                       args.down_t,
                       args.stride_t,
                       args.width,
                       args.depth,
                       args.dilation_growth_rate)

trans_encoder = trans.Text2Motion_Transformer(num_vq=args.nb_code,
                                embed_dim=1024,
                                clip_dim=args.clip_dim,
                                block_size=args.block_size,
                                num_layers=9,
                                n_head=16,
                                drop_out_rate=args.drop_out_rate,
                                fc_rate=args.ff_rate)

100%|████████████████████████████████████████| 338M/338M [00:02<00:00, 131MiB/s]


In [12]:
!bash dataset/prepare/download_model.sh
!bash dataset/prepare/download_extractor.sh
!bash dataset/prepare/download_smpl.sh

The pretrained model files will be stored in the 'pretrained' folder

Downloading...
From: https://drive.google.com/uc?id=1LaOvwypF-jM2Axnq5dc-Iuvv3w_G-WDE
To: /content/T2M-GPT/pretrained/VQTrans_pretrained.zip
100% 994M/994M [00:15<00:00, 64.6MB/s]
Archive:  VQTrans_pretrained.zip
  inflating: VQTransformer_corruption05/net_best_fid.pth  
  inflating: VQTransformer_corruption05/run.log  
  inflating: VQVAE/net_best_fid.pth  
  inflating: VQVAE/net_last.pth      
  inflating: VQVAE/run.log           
Cleaning

Downloading...
From: https://drive.google.com/uc?id=1o7RTDQcToJjTm9_mNWTyzvZvjTWpZfug
To: /content/T2M-GPT/checkpoints/t2m.zip
100% 1.22G/1.22G [00:20<00:00, 58.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1KNU8CsMAnxFrwopKBBkC8jEULGLPBHQp
To: /content/T2M-GPT/checkpoints/kit.zip
100% 705M/705M [00:13<00:00, 52.9MB/s]
Archive:  t2m.zip
   creating: t2m/
   creating: t2m/Comp_v6_KLD005/
   creating: t2m/Comp_v6_KLD005/meta/
  inflating: t2m/Comp_v6_KLD005/meta/mean.n

In [13]:
print ('loading checkpoint from {}'.format(args.resume_pth))
ckpt = torch.load(args.resume_pth, map_location='cpu')
net.load_state_dict(ckpt['net'], strict=True)
net.eval()
net.cuda()

print ('loading transformer checkpoint from {}'.format(args.resume_trans))
ckpt = torch.load(args.resume_trans, map_location='cpu')
trans_encoder.load_state_dict(ckpt['trans'], strict=True)
trans_encoder.eval()
trans_encoder.cuda()

mean = torch.from_numpy(np.load('./checkpoints/t2m/VQVAEV3_CB1024_CMT_H1024_NRES3/meta/mean.npy')).cuda()
std = torch.from_numpy(np.load('./checkpoints/t2m/VQVAEV3_CB1024_CMT_H1024_NRES3/meta/std.npy')).cuda()

loading checkpoint from pretrained/VQVAE/net_last.pth
loading transformer checkpoint from pretrained/VQTransformer_corruption05/net_best_fid.pth


# SMPL Class Setting

In [14]:
class joints2smpl:
    def __init__(self, num_frames, device_id, cuda=True):
        self.device = torch.device("cuda:" + str(device_id) if cuda else "cpu")
        # self.device = torch.device("cpu")
        self.batch_size = num_frames
        self.num_joints = 22  # for HumanML3D
        self.joint_category = "AMASS"
        self.num_smplify_iters = 150
        self.fix_foot = False
        print(config.SMPL_MODEL_DIR)
        smplmodel = smplx.create(config.SMPL_MODEL_DIR,
                                 model_type="smpl", gender="neutral", ext="pkl",
                                 batch_size=self.batch_size).to(self.device)

        # ## --- load the mean pose as original ----
        smpl_mean_file = config.SMPL_MEAN_FILE

        file = h5py.File(smpl_mean_file, 'r')
        self.init_mean_pose = torch.from_numpy(file['pose'][:]).unsqueeze(0).repeat(self.batch_size, 1).float().to(self.device)
        self.init_mean_shape = torch.from_numpy(file['shape'][:]).unsqueeze(0).repeat(self.batch_size, 1).float().to(self.device)
        self.cam_trans_zero = torch.Tensor([0.0, 0.0, 0.0]).unsqueeze(0).to(self.device)
        #

        # # #-------------initialize SMPLify
        self.smplify = SMPLify3D(smplxmodel=smplmodel,
                            batch_size=self.batch_size,
                            joints_category=self.joint_category,
                            num_iters=self.num_smplify_iters,
                            device=self.device)


    def npy2smpl(self, npy_path):
        out_path = npy_path.replace('.npy', '_rot.npy')
        motions = np.load(npy_path, allow_pickle=True)[None][0]
        # print_batch('', motions)
        n_samples = motions['motion'].shape[0]
        all_thetas = []
        for sample_i in tqdm(range(n_samples)):
            thetas, _ = self.joint2smpl(motions['motion'][sample_i].transpose(2, 0, 1))  # [nframes, njoints, 3]
            all_thetas.append(thetas.cpu().numpy())
        motions['motion'] = np.concatenate(all_thetas, axis=0)
        print('motions', motions['motion'].shape)

        print(f'Saving [{out_path}]')
        np.save(out_path, motions)
        exit()



    def joint2smpl(self, input_joints, init_params=None):
        _smplify = self.smplify # if init_params is None else self.smplify_fast
        pred_pose = torch.zeros(self.batch_size, 72).to(self.device)
        pred_betas = torch.zeros(self.batch_size, 10).to(self.device)
        pred_cam_t = torch.zeros(self.batch_size, 3).to(self.device)
        keypoints_3d = torch.zeros(self.batch_size, self.num_joints, 3).to(self.device)

        # run the whole seqs
        num_seqs = input_joints.shape[0]


        # joints3d = input_joints[idx]  # *1.2 #scale problem [check first]
        keypoints_3d = torch.Tensor(input_joints).to(self.device).float()

        # if idx == 0:
        if init_params is None:
            pred_betas = self.init_mean_shape
            pred_pose = self.init_mean_pose
            pred_cam_t = self.cam_trans_zero
        else:
            pred_betas = init_params['betas']
            pred_pose = init_params['pose']
            pred_cam_t = init_params['cam']

        if self.joint_category == "AMASS":
            confidence_input = torch.ones(self.num_joints)
            # make sure the foot and ankle
            if self.fix_foot == True:
                confidence_input[7] = 1.5
                confidence_input[8] = 1.5
                confidence_input[10] = 1.5
                confidence_input[11] = 1.5
        else:
            print("Such category not settle down!")

        new_opt_vertices, new_opt_joints, new_opt_pose, new_opt_betas, \
        new_opt_cam_t, new_opt_joint_loss = _smplify(
            pred_pose.detach(),
            pred_betas.detach(),
            pred_cam_t.detach(),
            keypoints_3d,
            conf_3d=confidence_input.to(self.device),
            # seq_ind=idx
        )

        thetas = new_opt_pose.reshape(self.batch_size, 24, 3)
        thetas = geometry.matrix_to_rotation_6d(geometry.axis_angle_to_matrix(thetas))  # [bs, 24, 6]
        root_loc = torch.tensor(keypoints_3d[:, 0])  # [bs, 3]
        root_loc = torch.cat([root_loc, torch.zeros_like(root_loc)], dim=-1).unsqueeze(1)  # [bs, 1, 6]
        thetas = torch.cat([thetas, root_loc], dim=1).unsqueeze(0).permute(0, 2, 3, 1)  # [1, 25, 6, 196]

        return thetas.clone().detach(), {'pose': new_opt_joints[0, :24].flatten().clone().detach(), 'betas': new_opt_betas.clone().detach(), 'cam': new_opt_cam_t.clone().detach(),
                                          'smpl_pose_72': new_opt_pose.clone().detach()}

# Inference Setting

In [15]:
def animate_3d_motion(joints, title=None,
                      folder_name = "anime", output_gif = "output.gif", duration=50):
    # Check if the folder already exists or not
    if not os.path.exists(folder_name):
        # Create the folder
        os.makedirs(folder_name)
        print(f"Folder '{folder_name}' created successfully.")
    else:
        print(f"Folder '{folder_name}' already exists.")

    # joints = xyz.detach().cpu().numpy().squeeze() # (time, num_joints, 3) No Batch
    data = joints.copy().reshape(len(joints), -1, 3) # (time, num_joints, 3)
    nb_joints = joints.shape[1]
    smpl_kinetic_chain = [[0, 11, 12, 13, 14, 15], [0, 16, 17, 18, 19, 20], [0, 1, 2, 3, 4], [3, 5, 6, 7], [3, 8, 9, 10]] if nb_joints == 21 else [[0, 2, 5, 8, 11], [0, 1, 4, 7, 10], [0, 3, 6, 9, 12, 15], [9, 14, 17, 19, 21], [9, 13, 16, 18, 20]]

    limits = 1000 if nb_joints == 21 else 2 # 2 or 1000
    MINS = data.min(axis=0).min(axis=0)
    MAXS = data.max(axis=0).max(axis=0)

    colors = ['red', 'blue', 'black', 'red', 'blue',
              'darkblue', 'darkblue', 'darkblue', 'darkblue', 'darkblue',
              'darkred', 'darkred', 'darkred', 'darkred', 'darkred']
    frame_number = data.shape[0]

    height_offset = MINS[1]
    data[:, :, 1] -= height_offset
    trajec = data[:, 0, [0, 2]] # [[0. 0.]] or tensor with shape (100, 2)

    data[..., 0] -= data[:, 0:1, 0]
    data[..., 2] -= data[:, 0:1, 2]

    def update(index):
        def init():
            ax.set_xlim(-limits, limits)
            ax.set_ylim(-limits, limits)
            ax.set_zlim(0, limits)
            ax.grid(b=False)

        def plot_xzPlane(minx, maxx, miny, minz, maxz):
            ## Plot a plane XZ
            verts = [
                [minx, miny, minz],
                [minx, miny, maxz],
                [maxx, miny, maxz],
                [maxx, miny, minz]
            ]
            xz_plane = Poly3DCollection([verts])
            xz_plane.set_facecolor((0.5, 0.5, 0.5, 0.5))
            ax.add_collection3d(xz_plane)

        fig = plt.figure(figsize=(480/96., 320/96.), dpi=96) if nb_joints == 21 else plt.figure(figsize=(10, 10), dpi=96) # Figure (960, 960)

        ax = fig.add_subplot(111, projection='3d')
        init()

        ax.view_init(elev=110, azim=-90) # Set the elevation and azimuth of the axes in degrees (not radians).
        ax.dist = 7.5

        plot_xzPlane(MINS[0] - trajec[index, 0], MAXS[0] - trajec[index, 0], 0, MINS[2] - trajec[index, 1],
                      MAXS[2] - trajec[index, 1])

        if index > 1:
            ax.plot3D(trajec[:index, 0] - trajec[index, 0], np.zeros_like(trajec[:index, 0]),
                      trajec[:index, 1] - trajec[index, 1], linewidth=1.0,
                      color='blue')

        for i, (chain, color) in enumerate(zip(smpl_kinetic_chain, colors)):
            if i < 5:
                linewidth = 4.0
            else:
                linewidth = 2.0
            ax.plot3D(data[index, chain, 0], data[index, chain, 1], data[index, chain, 2], linewidth=linewidth,
                      color=color)

        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_zticklabels([])
        frame_name = str(index)+".png"
        plt.savefig(os.path.join(folder_name, frame_name))
        plt.close()
        return

    for i in range(frame_number):
        update(i)

    ## All frames are generated.

    # List all PNG files in the folder
    png_files = [f for f in os.listdir(folder_name) if f.endswith(".png")]
    # Sort the files to ensure they are in the desired order
    png_files = sorted(png_files, key=lambda x: int(x.split(".")[0]))

    # Create a list to store the images
    images = []
    # Load each PNG file and append it to the images list
    for png_file in png_files:
        image_path = os.path.join(folder_name, png_file)
        img = Image.open(image_path)
        images.append(img)

    # Save the images as a GIF
    images[0].save(output_gif, save_all=True, append_images=images[1:], duration=duration, loop=0)

    try:
        # Use shutil.rmtree to delete the folder and its contents
        shutil.rmtree(folder_name)
        print(f"Folder '{folder_name}' and its contents have been deleted successfully.")
    except Exception as e:
        print(f"Error deleting folder: {e}")

# animate_3d_motion(joints = xyz.detach().cpu().numpy().squeeze())

# Inference

Expected output files for Patrick:
```
motion_{i}.npy
output_{i}.gif
motion_smpl_72_{i}.npy
```  
Patick's Note: __How to obtain the mean and standard deviation data (npy format)__  
These authors directly used the files extracted by T2M, because they are the mean and std for the pre-trained motion extractor. According to their code, these files are generated from the HumanML3D training dataset but with some rules, we can find their code below.  
https://github.com/EricGuo5513/text-to-motion/blob/main/data/dataset.py

In [17]:
#@title Prompts for XXXX
prompt_list = [
    "A person half kneel with one leg to work near the floor",
    "A person half squat to work near the floor",
    "A person raise both hands above his head and keep them there",
    "A person squat to carry up something",
    "A person move a box from left to right"
]

In [ ]:
for i, prompt in enumerate(prompt_list):
    n_output = 50 # number of outputs
    bookmark = 0 # bookmark to continue colab work

    text = clip.tokenize(prompt, truncate=True).cuda()
    feat_clip_text = clip_model.encode_text(text).float()

    save_folder_dir = "/content/drive/MyDrive/text2pose"
    folder_name = prompt.replace(" ", "_")
    folder_path = os.path.join(save_folder_dir, folder_name)
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    for j in tqdm(range(0, n_output), desc="processing..."):
        index_motion = trans_encoder.sample(feat_clip_text[0:1], True)
        pred_pose = net.forward_decoder(index_motion)

        pred_xyz = recover_from_ric((pred_pose*std+mean).float(), 22)
        xyz = pred_xyz.reshape(1, -1, 22, 3)
        motion = xyz.detach().cpu().numpy()

        ## motion xyz save
        motion_filename = 'motion_'+str(j)+'.npy'
        save_motion_dir = os.path.join(folder_path, motion_filename)
        np.save(save_motion_dir, motion)

        # frames, njoints, nfeats = xyz.detach().cpu().numpy().shape
        j2s = joints2smpl(num_frames=motion.shape[1], device_id=0, cuda=True)
        rot2xyz = Rotation2xyz(device=torch.device("cuda:0"))
        motion_tensor, opt_dict = j2s.joint2smpl(motion[0])

        ## smpl save
        smpl_filename = f'smpl_pose_72_{str(j)}.npy'
        save_smpl_dir = os.path.join(folder_path, smpl_filename)

        smpl_array = opt_dict['smpl_pose_72'].cpu().numpy()
        np.save(save_smpl_dir, smpl_array)

        ## gif save
        gif_filename = 'output_'+str(j)+'.gif'
        save_gif_dir = os.path.join(folder_path, gif_filename)
        animate_3d_motion(joints = motion.squeeze(), output_gif=save_gif_dir)

processing...:   0%|          | 0/50 [00:00<?, ?it/s]

./body_models/
Folder 'anime' created successfully.


processing...:   2%|▏         | 1/50 [01:59<1:37:52, 119.84s/it]

Folder 'anime' and its contents have been deleted successfully.
./body_models/
Folder 'anime' created successfully.


processing...:   4%|▍         | 2/50 [03:41<1:27:23, 109.24s/it]

Folder 'anime' and its contents have been deleted successfully.
./body_models/
Folder 'anime' created successfully.


processing...:   6%|▌         | 3/50 [05:30<1:25:20, 108.94s/it]

Folder 'anime' and its contents have been deleted successfully.
./body_models/
Folder 'anime' created successfully.


# Visualization

In [ ]:
#@title Mass Visualization
prompt = prompt_list[3]

folder_name = prompt.replace(" ", "_")
folder_dir = "/content/drive/MyDrive/text2pose"
folder_path = os.path.join(folder_dir, folder_name)

motion_files = [os.path.join(folder_path, filename) for filename in os.listdir(folder_path) if filename.startswith("motion_") and filename.endswith(".npy")]
motion_files.sort(key=lambda x: int(x.split("_")[-1].split(".")[0]))
motion_files = [np.load(motion) for motion in motion_files]
# motion file shape [1, # of frames, # of body joints (22), 3]

def animate_3d_motions(motions, grid_size=(3, 3), dist=3.0, ax_dist=7, title=None, duration=50, prompt=prompt):
    if not os.path.exists("temp_frames"):
        os.makedirs("temp_frames")
        print("Folder 'temp_frames' created successfully.")
    else:
        print("Folder 'temp_frames' already exists.")
    temp_folder_name = "temp_frames"

    motions = [motion.squeeze() for motion in motions]  # List of motions (numpy arrays)

    nb_frames = max(len(motion) for motion in motions)
    nb_motions = len(motions)
    nb_rows, nb_cols = grid_size

    # Pad shorter motions with the last frame to match the length of the longest motion
    for i in range(nb_motions):
        if len(motions[i]) < nb_frames:
            pad_frames = np.tile(motions[i][-1], (nb_frames - len(motions[i]), 1, 1))
            motions[i] = np.vstack((motions[i], pad_frames))

    height_axis = 2
    height_offset = min(motion[0, :, height_axis].min() for motion in motions)
    height_rescaled_motion = motions.copy()
    for i, motion in enumerate(motions):
        height_rescaled_motion[i][:,:,height_axis] -= height_offset

    grid_rows, grid_cols = grid_size
    distance_x, distance_y, distance_z = dist, dist, dist
    x_spare = dist/grid_cols

    data = np.zeros((height_rescaled_motion[0].shape[0], 22 * grid_rows * grid_cols, 3))

    # Iterate through the motions and place them in the grid
    for i, motion in enumerate(height_rescaled_motion):
        row_idx = i // grid_rows
        col_idx = i % grid_cols

        # Calculate the offset for each motion
        offset_x = col_idx * distance_x + row_idx * x_spare
        offset_y = 0 # row_idx * distance_z
        offset_z = row_idx * distance_z  # Assuming all motions are on the same z-plane
        # Apply the offset to the motion
        motion_with_offset = motion + np.array([offset_x, offset_y, offset_z])
        # Insert the motion into the data array
        data[:, i * 22:(i + 1) * 22, :] = motion_with_offset

    frame_number = data.shape[0]
    nb_joints = data.shape[1]
    smpl_kinetic_chain = [[0, 11, 12, 13, 14, 15], [0, 16, 17, 18, 19, 20], [0, 1, 2, 3, 4], [3, 5, 6, 7], [3, 8, 9, 10]] if nb_joints == 21 \
                          else [[0, 2, 5, 8, 11], [0, 1, 4, 7, 10], [0, 3, 6, 9, 12, 15], [9, 14, 17, 19, 21], [9, 13, 16, 18, 20]]
    num_persons = len(motions)
    smpl_kinetic_chain_list = []

    for p in range(num_persons):
        for group in smpl_kinetic_chain:
            temp = []
            for joint_idx in group:
                temp.append(joint_idx+22*p)
            smpl_kinetic_chain_list.append(temp)

    colors = ["red", "blue", "black", "red", "blue"]*num_persons # color order: [left_leg, right_leg, body, left_arm, right_arm]
    verts = [[-1, 0, -1],
             [-1, 0, dist*grid_rows],
             [dist*grid_rows, 0, dist*grid_rows+1],
             [dist*grid_rows, 0, -1]]

    for frame in range(frame_number):
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax.view_init(elev=95, azim=-90)
        ax.dist = ax_dist
        ax.set_xlim(-1, dist*grid_rows)
        ax.set_ylim(0, 2)
        ax.set_zlim(-1, dist*grid_rows)
        ax.set_box_aspect([dist*grid_rows+1, 2, dist*grid_rows+1])
        ax.grid(b=False)
        # ax.set_xlabel('X Label')
        # ax.set_ylabel('Y Label')
        # ax.set_zlabel('Z Label')
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_zticklabels([])

        xz_plane = Poly3DCollection([verts])
        xz_plane.set_facecolor((0.5, 0.5, 0.5, 0.2))
        ax.add_collection3d(xz_plane)
        for chain, color in zip(smpl_kinetic_chain_list, colors):
            ax.plot3D(data[frame, chain, 0], data[frame, chain, 1], data[frame, chain, 2], linewidth=2, color=color)
        
        frame_name = str(frame)+".png"
        plt.savefig(os.path.join(temp_folder_name, frame_name))
        plt.close()

    png_files = [f for f in os.listdir(temp_folder_name) if f.endswith(".png")]
    # Sort the files to ensure they are in the desired order
    png_files = sorted(png_files, key=lambda x: int(x.split(".")[0]))

    # Create a list to store the images
    images = []
    # Load each PNG file and append it to the images list
    for png_file in png_files:
        image_path = os.path.join(temp_folder_name, png_file)
        img = Image.open(image_path)
        images.append(img)

    file_name = prompt.replace(" ", "_")+"_mass.gif"
    file_path = os.path.join(folder_dir, file_name)
    # Save the images as a GIF
    images[0].save(file_path, save_all=True, append_images=images[1:], duration=duration, loop=0)

    try:
        # Use shutil.rmtree to delete the folder and its contents
        shutil.rmtree(temp_folder_name)
        print(f"Folder '{temp_folder_name}' and its contents have been deleted successfully.")
    except Exception as e:
        print(f"Error deleting folder: {e}")

motion_files = motion_files[:9]
animate_3d_motions(motion_files, grid_size=(3, 3), dist=3, ax_dist=7, prompt=prompt)